In [24]:
# This script parses out the lpjml crop mask files and writes the data out to disk for use by the 
# parse_lpjml_output.ipynb script.

From Sebastian - 

You should find a new subdirectory "landuse_patterns" in the PIK cloud
share.
This directory contains 2 LPJ input files and 2 tools

- cellarea.clm: input file with area of each cell in hectare
- mockup_lu.clm: input file with harvested area per crop (and
rainfed/irrigated) in fraction -> multiply with cellarea.clm to get
harvested areas in hectares

You can use
./headersize cellarea.clm
to get the header size in bytes of the input file.

You can use
./printclm -data cellarea.clm
to print out the information contained in the header.

This gives you the necessary information to read in the files.

In [25]:
import struct
import numpy as np
import pickle

output of ./headersize cellarea.clm call is 51 (on my linux box)

In [26]:
HEADER_BYTES = 51

In [27]:
f = open("data/landuse_patterns/cellarea.clm","rb")
_ = f.read(HEADER_BYTES)

In [28]:
# Get hectare values for cells

hectares = []
# This will error out when done reading, hence the try/except
while True:
    try:
        four_byes = f.read(4)
        float_value = struct.unpack("<f", four_byes)
        hectares.append(float_value[0])
    except:
        break

In [29]:
# Make sure this is true - number of cells should match 
len(hectares) == 67420

True

crop_mask.txt was the output of ./printclm cellarea.clm (on my linux box)
It has a header and then a % value per cell per crop-hydro but only for year 2000

In [30]:
# This parses out the crop_mask.txt file output described above.
start = False
cells = []
for line in open("data/landuse_patterns/crop_mask.txt"):
    if not start:
        if line.startswith("Year"):
            start = True
            continue
    else:
        cell_num, vals = line.split(":")
        vals = vals.split()
        vals = list(map(lambda x: float(x.strip()),vals))
        cells.append(vals)

In [31]:
data = np.array(cells)

In [32]:
# A row per cell where the row contains the % for the corresponding crop-hydro value
data.shape

(67420, 32)

In [33]:
# Dump this data to disk for use by the other script
pickle.dump( data, open( "data/landuse_patterns/crop_mask.npy", "wb" ) )
mask_data = pickle.load(open("crop_mask.npy", "rb"))

In [34]:
# Dump the hectare data to disk for use by the other script
pickle.dump( hectares, open( "data/landuse_patterns/hectares.npy", "wb" ) )
hectares = pickle.load(open("hectares.npy", "rb"))

In [35]:
mask_data.shape

(67420, 32)

In [36]:
len(hectares)

67420